In [1]:
# Nous allons écrire une implémentation simple d’un algorithme de recommandation ALS afin de permettre la recommendation des produit pour les clients
# Base de donnée utilisée : reviews_Grocery_and_Gourmet_Food_5.json
# Les algorithmes à utiliser sont classés sous la catégorie "Collaborative filtering", ou filtrage collaboratif en français
# et qui regroupe un ensemble de méthodes visant à construire des systèmes de recommendation utilisant les opinions et
# évaluations d'un groupe pour aider l'individu.
# L'algorithme ALS (Alternating Least Squares) est le modèle que nous utiliserons pour ajuster nos données et trouver des similitudes et il est disponible sous Spark.


#-----------------------------------1-Importation des modules necessaires---------------------------------
import findspark
findspark.init('C:/Users/Mohammed/Desktop/new_folder/spark-2.4.0-bin-hadoop2.7/spark-2.4.0-bin-hadoop2.7')
#pyspark.sql qui contient des fonctions pour la lecture des fichiers de données
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import udf,col
#pyspqrk.ml qui contient des fontions pour entrainer et evaluer notre modele
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer

In [2]:
#----------------------------------2-Lire les données en tant que dataframe------------------------------
#En créant
# SparkSession fournit un point d'entrée unique pour interagir avec la fonctionnalité Spark
spark = SparkSession.builder.master('local').appName("Collaborative filtring").config("spark.executor.memory", "1gb").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
#lire les données en utilisant read.json qui permet de lire les fichiers de type json
df = sqlContext.read.json('Grocery_and_Gourmet_Food_5.json')
df.show(5)

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|   reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|616719923X| [0, 0]|    4.0|Just another flav...| 06 1, 2013|A1VEELTKS8NLZB|Amazon Customer|          Good Taste|    1370044800|
|616719923X| [0, 1]|    3.0|I bought this on ...|05 19, 2014|A14R9XMZVJ6INB|        amf0001|3.5 stars,  sadly...|    1400457600|
|616719923X| [3, 4]|    4.0|Really good. Grea...| 10 8, 2013|A27IQHDZFQFNGG|        Caitlin|                Yum!|    1381190400|
|616719923X| [0, 0]|    5.0|I had never had i...|05 20, 2013|A31QY5TASILE89|   DebraDownSth|Unexpected flavor...|    1369008000|
|616719923X| [1, 2]|    4.0|I've been looking...|05 26, 2013|A2LWK003FFMCI5|       Diana X.|Not a

In [3]:
#----------------------------------3-Data Cleaning------------------------------
#On va supprimer quelques colonnes qu'on juge unitile à notre analyse
df = df.select('asin', 'helpful', 'overall', 'reviewerID', 'reviewerName')
df.show()

+----------+-------+-------+--------------+--------------------+
|      asin|helpful|overall|    reviewerID|        reviewerName|
+----------+-------+-------+--------------+--------------------+
|616719923X| [0, 0]|    4.0|A1VEELTKS8NLZB|     Amazon Customer|
|616719923X| [0, 1]|    3.0|A14R9XMZVJ6INB|             amf0001|
|616719923X| [3, 4]|    4.0|A27IQHDZFQFNGG|             Caitlin|
|616719923X| [0, 0]|    5.0|A31QY5TASILE89|        DebraDownSth|
|616719923X| [1, 2]|    4.0|A2LWK003FFMCI5|            Diana X.|
|616719923X| [0, 1]|    4.0|A1NZJTY0BAA2SK|           Elizabeth|
|616719923X| [1, 2]|    3.0| AA95FYFIP38RM|Emily Veinglory "...|
|616719923X| [2, 3]|    5.0|A3FIVHUOGMUMPK|           greenlife|
|616719923X| [0, 0]|    5.0|A27FSPAMTQF1J8|              Japhyl|
|616719923X|[0, 10]|    1.0|A33NXNZ79H5K51|         Jean M "JM"|
|616719923X| [6, 8]|    5.0|A220GN2X2R47JE|              Jeremy|
|616719923X| [2, 3]|    5.0|A3C5Z05IKSSFB9|M. Magpoc "malias...|
|616719923X| [0, 0]|    5

In [4]:
#----------------------------------4-Changement du nom de certain attribut------------------------------
df=df.withColumn('overall', df['overall'].cast(FloatType()))
df = df.selectExpr("asin as productID", "overall as rating", "helpful as helpful",\
                   "reviewerID as reviewerID", "reviewerName as reviewerName")
df.show()

+----------+------+-------+--------------+--------------------+
| productID|rating|helpful|    reviewerID|        reviewerName|
+----------+------+-------+--------------+--------------------+
|616719923X|   4.0| [0, 0]|A1VEELTKS8NLZB|     Amazon Customer|
|616719923X|   3.0| [0, 1]|A14R9XMZVJ6INB|             amf0001|
|616719923X|   4.0| [3, 4]|A27IQHDZFQFNGG|             Caitlin|
|616719923X|   5.0| [0, 0]|A31QY5TASILE89|        DebraDownSth|
|616719923X|   4.0| [1, 2]|A2LWK003FFMCI5|            Diana X.|
|616719923X|   4.0| [0, 1]|A1NZJTY0BAA2SK|           Elizabeth|
|616719923X|   3.0| [1, 2]| AA95FYFIP38RM|Emily Veinglory "...|
|616719923X|   5.0| [2, 3]|A3FIVHUOGMUMPK|           greenlife|
|616719923X|   5.0| [0, 0]|A27FSPAMTQF1J8|              Japhyl|
|616719923X|   1.0|[0, 10]|A33NXNZ79H5K51|         Jean M "JM"|
|616719923X|   5.0| [6, 8]|A220GN2X2R47JE|              Jeremy|
|616719923X|   5.0| [2, 3]|A3C5Z05IKSSFB9|M. Magpoc "malias...|
|616719923X|   5.0| [0, 0]| AHA6G4IMEMAJ

In [5]:
#----------------------code une colonne de chaîne de caractère en une colonne d'index-------------------
#Ajouter d'autre colonnes qui vont faciliter notre analyse, aussi transformer 
#le type de quelques colonnes string-->int
product_indexer = StringIndexer(inputCol = "productID", outputCol = "productID_new")
reviewer_indexer = StringIndexer(inputCol = "reviewerID", outputCol = "reviewerID_new")

df2 = product_indexer.fit(df).transform(df)
df2 = reviewer_indexer.fit(df2).transform(df2)

df2 = df2.withColumn('reviewerID_new', df2['reviewerID_new'].cast(IntegerType()))
df2 = df2.withColumn('productID_new', df2['productID_new'].cast(IntegerType()))
df2.show()

+----------+------+-------+--------------+--------------------+-------------+--------------+
| productID|rating|helpful|    reviewerID|        reviewerName|productID_new|reviewerID_new|
+----------+------+-------+--------------+--------------------+-------------+--------------+
|616719923X|   4.0| [0, 0]|A1VEELTKS8NLZB|     Amazon Customer|         1870|          6633|
|616719923X|   3.0| [0, 1]|A14R9XMZVJ6INB|             amf0001|         1870|           783|
|616719923X|   4.0| [3, 4]|A27IQHDZFQFNGG|             Caitlin|         1870|          8454|
|616719923X|   5.0| [0, 0]|A31QY5TASILE89|        DebraDownSth|         1870|          5294|
|616719923X|   4.0| [1, 2]|A2LWK003FFMCI5|            Diana X.|         1870|          7498|
|616719923X|   4.0| [0, 1]|A1NZJTY0BAA2SK|           Elizabeth|         1870|         12719|
|616719923X|   3.0| [1, 2]| AA95FYFIP38RM|Emily Veinglory "...|         1870|          9071|
|616719923X|   5.0| [2, 3]|A3FIVHUOGMUMPK|           greenlife|       

In [6]:
#------------------------Implémentation de l'algorithme ALS--------------------------
#On commencera par diviser notre ensemble de données en deux parties : Données de test (20%) et donnée d'apprentissage (80%)
(training, test) = df2.randomSplit([0.8, 0.2])
# Definition du modèle
als = ALS(maxIter = 5, regParam = 0.01, userCol = "reviewerID_new", itemCol = "productID_new"\
         , ratingCol = "rating", coldStartStrategy = "drop")

model = als.fit(training)

# Faire des prediction sur les données de test
predictions = model.transform(test)

# Evaluer le modèle
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
#Root Mean Square Error pour la distribution de données par rapport à la line de regression
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 4.704192262959549


In [7]:
#Generer les top 10 produits recommendé pour un client 
userRecs = model.recommendForAllUsers(10)
userRecs.show()

+--------------+--------------------+
|reviewerID_new|     recommendations|
+--------------+--------------------+
|          1580|[[1678, 35.47259]...|
|          4900|[[3332, 19.778898...|
|          5300|[[1068, 8.514068]...|
|          6620|[[5987, 20.10723]...|
|          7240|[[6125, 8.254153]...|
|          7340|[[1791, 10.923169...|
|          7880|[[2206, 7.0005302...|
|          9900|[[1666, 5.737188]...|
|         12940|[[2546, 9.567051]...|
|         13840|[[1959, 15.273684...|
|         14450|[[3016, 13.69047]...|
|         14570|[[2548, 9.119081]...|
|           471|[[2301, 16.142128...|
|          1591|[[1959, 23.958126...|
|          4101|[[2394, 29.692163...|
|         11141|[[2431, 10.337481...|
|          1342|[[5009, 12.26683]...|
|          2122|[[2536, 21.467627...|
|          2142|[[3194, 12.762062...|
|          7982|[[5961, 8.691798]...|
+--------------+--------------------+
only showing top 20 rows

